# Truck Analysis
Lets do some sentiment analysis on our food trucks

In [ ]:
USE DATABASE tb_voc;
-- schema to hold all tables, functions, stored procedures
CREATE SCHEMA IF NOT EXISTS data;
-- stage for udf function files, other data files
CREATE SCHEMA IF NOT EXISTS stages;

In [ ]:
select * 
from tb_voc.analytics.truck_reviews_v
limit 5;

In [ ]:
-- Stored Procedure to build the sentiment out of reviews
CREATE OR REPLACE PROCEDURE data.get_review_sentiment()
RETURNS TABLE (
    truck_id int, review varchar, sentiment_score float
)
LANGUAGE SQL AS
$$
DECLARE
  res RESULTSET;
BEGIN
  res := ( 
    SELECT
        truck_id,
        review,
        ROUND(SNOWFLAKE.CORTEX.SENTIMENT(review),2) AS sentiment
    FROM tb_voc.analytics.truck_reviews_v
    WHERE date_part('year', date) = 2024
);
  RETURN TABLE(res);
END;
$$;

In [ ]:
__database = "tb_voc"
__schema = "analytics"
__data_schema = "data"
__stages_schema = "stages"
__warehouse = "tasty_ds_wh"
__udf_stage = "udfs"

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.core import CreateMode, Root
from snowflake.core.stage import Stage

session = get_active_session()
root = Root(session)
function_stage = Stage(name=__udf_stage)
function_stage: Stage = (
    root.databases[__database]
    .schemas[__stages_schema]
    .stages.create(
        function_stage,
        mode=CreateMode.if_not_exists,
    )
)

In [ ]:
from snowflake.snowpark.functions import udf
@udf(
    name=f"{__database}.{__data_schema}.classify_sentiment",
    is_permanent=True,
    packages=["snowflake-telemetry-python"],
    stage_location=f"{__database}.{__stages_schema}.{__udf_stage}",
    replace=True,
)
def classify_sentiment(sentiment_score: float) -> str:
    """Classify sentiment as positive,neutral or negative based on the score."""
    import logging

    import snowflake.telemetry as telemetry

    logging.debug("Classifying sentiment score")

    telemetry.set_span_attribute("executing", "classify_sentiment")
    logging.debug(f"Classifying sentiment score {sentiment_score:.2f}")

    if sentiment_score < -0.5:
        logging.debug("Sentiment is negative")
        return "negative"
    elif sentiment_score >= 0.5 and sentiment_score <= 1.0:
        logging.debug("Sentiment is positive")
        return "positive"
    else:
        logging.debug("Sentiment is neutral")
        return "netural"

In [ ]:
set num_of_rows=20;

SELECT
    truck_id,
    review,
    sentiment_score,
    {{__database}}.{{__data_schema}}.classify_sentiment(sentiment_score) as sentiment_class
FROM table({{__database}}.{{__data_schema}}.get_review_sentiment())
LIMIT $num_of_rows

## Serverless Tasks

## Serverless Alerts

## Notifications